In [1]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM




In [2]:
# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
logging.basicConfig(level=logging.INFO)


In [24]:
# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

03/06/2020 09:00:10 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /home/srtp/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


In [5]:
test_int_a=1

In [6]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from random import randrange
import torch.nn.functional as F

In [7]:
text ='what is a pug'

In [8]:
zz = tokenizer.tokenize(text)


In [25]:
class BertLayerNorm(nn.Module):
        def __init__(self, hidden_size, eps=1e-12):
            """Construct a layernorm module in the TF style (epsilon inside the square root).
            """
            super(BertLayerNorm, self).__init__()
            self.weight = nn.Parameter(torch.ones(hidden_size))
            self.bias = nn.Parameter(torch.zeros(hidden_size))
            self.variance_epsilon = eps

        def forward(self, x):
            u = x.mean(-1, keepdim=True)
            s = (x - u).pow(2).mean(-1, keepdim=True)
            x = (x - u) / torch.sqrt(s + self.variance_epsilon)
            return self.weight * x + self.bias
        

class BertForSequenceClassification(nn.Module):
    """BERT model for classification.
    This module is composed of the BERT model with a linear layer on top of
    the pooled output.
    Params:
        `config`: a BertConfig class instance with the configuration to build a new model.
        `num_labels`: the number of classes for the classifier. Default = 2.
    Inputs:
        `input_ids`: a torch.LongTensor of shape [batch_size, sequence_length]
            with the word token indices in the vocabulary. Items in the batch should begin with the special "CLS" token. (see the tokens preprocessing logic in the scripts
            `extract_features.py`, `run_classifier.py` and `run_squad.py`)
        `token_type_ids`: an optional torch.LongTensor of shape [batch_size, sequence_length] with the token
            types indices selected in [0, 1]. Type 0 corresponds to a `sentence A` and type 1 corresponds to
            a `sentence B` token (see BERT paper for more details).
        `attention_mask`: an optional torch.LongTensor of shape [batch_size, sequence_length] with indices
            selected in [0, 1]. It's a mask to be used if the input sequence length is smaller than the max
            input sequence length in the current batch. It's the mask that we typically use for attention when
            a batch has varying length sentences.
        `labels`: labels for the classification output: torch.LongTensor of shape [batch_size]
            with indices selected in [0, ..., num_labels].
    Outputs:
        if `labels` is not `None`:
            Outputs the CrossEntropy classification loss of the output with the labels.
        if `labels` is `None`:
            Outputs the classification logits of shape [batch_size, num_labels].
    Example usage:
    ```python
    # Already been converted into WordPiece token ids
    input_ids = torch.LongTensor([[31, 51, 99], [15, 5, 0]])
    input_mask = torch.LongTensor([[1, 1, 1], [1, 1, 0]])
    token_type_ids = torch.LongTensor([[0, 0, 1], [0, 1, 0]])
    config = BertConfig(vocab_size_or_config_json_file=32000, hidden_size=768,
        num_hidden_layers=12, num_attention_heads=12, intermediate_size=3072)
    num_labels = 2
    model = BertForSequenceClassification(config, num_labels)
    logits = model(input_ids, token_type_ids, input_mask)
    ```
    """
    def __init__(self, num_labels=2):
        super(BertForSequenceClassification, self).__init__()
        self.num_labels = num_labels
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, num_labels)
        nn.init.xavier_normal_(self.classifier.weight)
    def forward(self, input_ids, token_type_ids=None, attention_mask=None, labels=None):
        _, pooled_output = self.bert(input_ids, token_type_ids, attention_mask, output_all_encoded_layers=False)
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)

        return logits
    def freeze_bert_encoder(self):
        for param in self.bert.parameters():
            param.requires_grad = False
    
    def unfreeze_bert_encoder(self):
        for param in self.bert.parameters():
            param.requires_grad = True

In [26]:
from pytorch_pretrained_bert import BertConfig

config = BertConfig(vocab_size_or_config_json_file=32000, hidden_size=768,
        num_hidden_layers=12, num_attention_heads=12, intermediate_size=3072)



In [27]:
num_labels = 2
model = BertForSequenceClassification(num_labels)



03/06/2020 09:00:28 - INFO - pytorch_pretrained_bert.modeling -   loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at /home/srtp/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
03/06/2020 09:00:28 - INFO - pytorch_pretrained_bert.modeling -   extracting archive file /home/srtp/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /tmp/tmpgxdgwxu_
03/06/2020 09:00:31 - INFO - pytorch_pretrained_bert.modeling -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "

In [28]:
# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([tokenizer.convert_tokens_to_ids(zz)])

logits = model(tokens_tensor)

In [29]:
logits

tensor([[-1.2959,  0.0543]], grad_fn=<AddmmBackward>)

In [30]:
import pandas as pd

dat = pd.read_csv('IMDB Dataset.csv')

In [31]:
dat.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [32]:
zz = tokenizer.tokenize(dat.review[1])
z1z = tokenizer.tokenize(dat.review[3])

[zz,z1z ]

[['a',
  'wonderful',
  'little',
  'production',
  '.',
  '<',
  'br',
  '/',
  '>',
  '<',
  'br',
  '/',
  '>',
  'the',
  'filming',
  'technique',
  'is',
  'very',
  'una',
  '##ss',
  '##uming',
  '-',
  'very',
  'old',
  '-',
  'time',
  '-',
  'bbc',
  'fashion',
  'and',
  'gives',
  'a',
  'comforting',
  ',',
  'and',
  'sometimes',
  'discomfort',
  '##ing',
  ',',
  'sense',
  'of',
  'realism',
  'to',
  'the',
  'entire',
  'piece',
  '.',
  '<',
  'br',
  '/',
  '>',
  '<',
  'br',
  '/',
  '>',
  'the',
  'actors',
  'are',
  'extremely',
  'well',
  'chosen',
  '-',
  'michael',
  'sheen',
  'not',
  'only',
  '"',
  'has',
  'got',
  'all',
  'the',
  'polar',
  '##i',
  '"',
  'but',
  'he',
  'has',
  'all',
  'the',
  'voices',
  'down',
  'pat',
  'too',
  '!',
  'you',
  'can',
  'truly',
  'see',
  'the',
  'seam',
  '##less',
  'editing',
  'guided',
  'by',
  'the',
  'references',
  'to',
  'williams',
  "'",
  'diary',
  'entries',
  ',',
  'not',
  'only

In [33]:
zzz = tokenizer.convert_tokens_to_ids(zz)
zzzz = tokenizer.convert_tokens_to_ids(z1z)

In [34]:
zzz

[1037,
 6919,
 2210,
 2537,
 1012,
 1026,
 7987,
 1013,
 1028,
 1026,
 7987,
 1013,
 1028,
 1996,
 7467,
 6028,
 2003,
 2200,
 14477,
 4757,
 24270,
 1011,
 2200,
 2214,
 1011,
 2051,
 1011,
 4035,
 4827,
 1998,
 3957,
 1037,
 16334,
 1010,
 1998,
 2823,
 17964,
 2075,
 1010,
 3168,
 1997,
 15650,
 2000,
 1996,
 2972,
 3538,
 1012,
 1026,
 7987,
 1013,
 1028,
 1026,
 7987,
 1013,
 1028,
 1996,
 5889,
 2024,
 5186,
 2092,
 4217,
 1011,
 2745,
 20682,
 2025,
 2069,
 1000,
 2038,
 2288,
 2035,
 1996,
 11508,
 2072,
 1000,
 2021,
 2002,
 2038,
 2035,
 1996,
 5755,
 2091,
 6986,
 2205,
 999,
 2017,
 2064,
 5621,
 2156,
 1996,
 25180,
 3238,
 9260,
 8546,
 2011,
 1996,
 7604,
 2000,
 3766,
 1005,
 9708,
 10445,
 1010,
 2025,
 2069,
 2003,
 2009,
 2092,
 4276,
 1996,
 3666,
 2021,
 2009,
 2003,
 1037,
 27547,
 2135,
 2517,
 1998,
 2864,
 3538,
 1012,
 1037,
 3040,
 3993,
 2537,
 2055,
 2028,
 1997,
 1996,
 2307,
 3040,
 1005,
 1055,
 1997,
 4038,
 1998,
 2010,
 2166,
 1012,
 1026,
 7987,
 101

In [35]:

tokens_tensor = torch.tensor([zzz,zzz])


In [36]:
tokens_tensor

tensor([[ 1037,  6919,  2210,  2537,  1012,  1026,  7987,  1013,  1028,  1026,
          7987,  1013,  1028,  1996,  7467,  6028,  2003,  2200, 14477,  4757,
         24270,  1011,  2200,  2214,  1011,  2051,  1011,  4035,  4827,  1998,
          3957,  1037, 16334,  1010,  1998,  2823, 17964,  2075,  1010,  3168,
          1997, 15650,  2000,  1996,  2972,  3538,  1012,  1026,  7987,  1013,
          1028,  1026,  7987,  1013,  1028,  1996,  5889,  2024,  5186,  2092,
          4217,  1011,  2745, 20682,  2025,  2069,  1000,  2038,  2288,  2035,
          1996, 11508,  2072,  1000,  2021,  2002,  2038,  2035,  1996,  5755,
          2091,  6986,  2205,   999,  2017,  2064,  5621,  2156,  1996, 25180,
          3238,  9260,  8546,  2011,  1996,  7604,  2000,  3766,  1005,  9708,
         10445,  1010,  2025,  2069,  2003,  2009,  2092,  4276,  1996,  3666,
          2021,  2009,  2003,  1037, 27547,  2135,  2517,  1998,  2864,  3538,
          1012,  1037,  3040,  3993,  2537,  2055,  

In [37]:
logits = model(tokens_tensor)
logits

tensor([[-0.8152,  0.4565],
        [-0.4468,  1.8472]], grad_fn=<AddmmBackward>)

In [38]:
import torch.nn.functional as F

F.softmax(logits,dim=1)

tensor([[0.2190, 0.7810],
        [0.0916, 0.9084]], grad_fn=<SoftmaxBackward>)

In [39]:
from sklearn.model_selection import train_test_split
X = dat['review']
y = dat['sentiment']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42)


In [40]:
X_train = X_train.values.tolist()
X_test = X_test.values.tolist()

y_train = pd.get_dummies(y_train).values.tolist()
y_test = pd.get_dummies(y_test).values.tolist()


In [41]:

max_seq_length = 256
class text_dataset(Dataset):
    def __init__(self,x_y_list, transform=None):
        
        self.x_y_list = x_y_list
        self.transform = transform
        
    def __getitem__(self,index):
        
        tokenized_review = tokenizer.tokenize(self.x_y_list[0][index])
        
        if len(tokenized_review) > max_seq_length:
            tokenized_review = tokenized_review[:max_seq_length]
            
        ids_review  = tokenizer.convert_tokens_to_ids(tokenized_review)

        padding = [0] * (max_seq_length - len(ids_review))
        
        ids_review += padding
        
        assert len(ids_review) == max_seq_length
        
        #print(ids_review)
        ids_review = torch.tensor(ids_review)
        
        sentiment = self.x_y_list[1][index] # color        
        list_of_labels = [torch.from_numpy(np.array(sentiment))]
        
        
        return ids_review, list_of_labels[0]
    
    def __len__(self):
        return len(self.x_y_list[0])

In [42]:

batch_size = 16

train_lists = [X_train, y_train]
test_lists = [X_test, y_test]

training_dataset = text_dataset(x_y_list = train_lists )

test_dataset = text_dataset(x_y_list = test_lists )

dataloaders_dict = {'train': torch.utils.data.DataLoader(training_dataset, batch_size=batch_size, shuffle=True, num_workers=0),
                   'val':torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
                   }
dataset_sizes = {'train':len(train_lists[0]),
                'val':len(test_lists[0])}

device = torch.device( "cpu")
print(device)

cpu


In [54]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()
    print('starting')
    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = 100

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            
            sentiment_corrects = 0
            
            
            # Iterate over data.
            for inputs, sentiment in dataloaders_dict[phase]:
                #inputs = inputs
                #print(len(inputs),type(inputs),inputs)
                #inputs = torch.from_numpy(np.array(inputs)).to(device) 
                inputs = inputs.to(device) 

                sentiment = sentiment.to(device)
                
                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    #print(inputs)
                    outputs = model(inputs)

                    outputs = F.softmax(outputs,dim=1)
                    
                    loss = criterion(outputs, torch.max(sentiment.float(), 1)[1])
                    # backward + optimize only if in training phase
                    if phase == 'train':
                        
                        loss.backward()
                        optimizer.step()
                        print("training")

                # statistics
                running_loss += loss.item() * inputs.size(0)
                print("current loss:",loss.item() * inputs.size(0))

                
                sentiment_corrects += torch.sum(torch.max(outputs, 1)[1] == torch.max(sentiment, 1)[1])

                
            epoch_loss = running_loss / dataset_sizes[phase]

            
            sentiment_acc = sentiment_corrects.double() / dataset_sizes[phase]

            print('{} total loss: {:.4f} '.format(phase,epoch_loss ))
            print('{} sentiment_acc: {:.4f}'.format(
                phase, sentiment_acc))

            if phase == 'val' and epoch_loss < best_loss:
                print('saving with loss of {}'.format(epoch_loss),
                      'improved over previous {}'.format(best_loss))
                best_loss = epoch_loss
                best_model_wts = copy.deepcopy(model.state_dict())
                torch.save(model.state_dict(), 'bert_model_test.pth')


        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(float(best_loss)))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [55]:
model.to(device)

#model.freeze_bert_encoder()
#model.classifier.weight.requires_grad = True
model

#    def freeze_bert_encoder(self):
#        for param in self.bert.parameters():
#            param.requires_grad = False
#    
#    def unfreeze_bert_encoder(self):
#        for param in self.bert.parameters():
#            param.requires_grad = True

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1)
            )
          )
          (intermediate): BertInterme

In [56]:
lrlast = .001
lrmain = .00001
optim1 = optim.Adam(
    [
        {"params":model.bert.parameters(),"lr": lrmain},
        {"params":model.classifier.parameters(), "lr": lrlast},
       
   ])

#optim1 = optim.Adam(model.parameters(), lr=0.001)#,momentum=.9)
# Observe that all parameters are being optimized
optimizer_ft = optim1
criterion = nn.CrossEntropyLoss()

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=3, gamma=0.1)


In [ ]:
model_ft1 = train_model(model, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=5)


starting
Epoch 0/4
----------
training
current loss: 11.638134956359863
training
current loss: 9.375847816467285
training
current loss: 9.472310066223145
training
current loss: 9.69799518585205
training
current loss: 10.83901596069336
training
current loss: 10.129119873046875
training
current loss: 11.161550521850586
training
current loss: 9.692031860351562
training
current loss: 8.49592113494873
training
current loss: 11.329404830932617
training
current loss: 8.406257629394531
training
current loss: 10.83920669555664
training
current loss: 10.086922645568848
training
current loss: 10.588093757629395
training
current loss: 10.32459545135498
training
current loss: 9.913704872131348
training
current loss: 11.75322437286377
training
current loss: 10.181161880493164
training
current loss: 8.828126907348633
training
current loss: 8.242478370666504
training
current loss: 8.881961822509766
training
current loss: 6.912471771240234
training
current loss: 7.8965959548950195
training
current loss

training
current loss: 8.710936546325684
training
current loss: 5.064635276794434
training
current loss: 7.431754112243652
training
current loss: 6.528200626373291
training
current loss: 7.495674133300781
training
current loss: 5.717403411865234
training
current loss: 6.933420181274414
training
current loss: 7.253873825073242
training
current loss: 5.704046726226807
training
current loss: 7.1401591300964355
training
current loss: 5.4821929931640625
training
current loss: 9.383604049682617
training
current loss: 7.231834411621094
training
current loss: 6.4801130294799805
training
current loss: 7.199197292327881
training
current loss: 5.137447357177734
training
current loss: 7.814896106719971
training
current loss: 7.687524795532227
training
current loss: 8.139082908630371
training
current loss: 8.423921585083008
training
current loss: 6.0358476638793945
training
current loss: 6.71282958984375
training
current loss: 10.62513256072998
training
current loss: 5.9373250007629395
training
cur

training
current loss: 6.039558410644531
training
current loss: 7.396637916564941
training
current loss: 7.93662166595459
training
current loss: 9.012825965881348
training
current loss: 5.112208366394043
training
current loss: 6.032895088195801
training
current loss: 5.02143669128418
training
current loss: 6.985262393951416
training
current loss: 6.159646034240723
training
current loss: 6.66090726852417
training
current loss: 7.011622428894043
training
current loss: 7.07234525680542
training
current loss: 5.761606693267822
training
current loss: 7.877546787261963
training
current loss: 7.01157283782959
training
current loss: 5.646257400512695
training
current loss: 6.043929100036621
training
current loss: 5.300237655639648
training
current loss: 6.842912197113037
training
current loss: 5.022723197937012
training
current loss: 6.412811756134033
training
current loss: 7.003622531890869
training
current loss: 6.025242805480957
training
current loss: 5.285910129547119
training
current loss

training
current loss: 6.021851062774658
training
current loss: 6.1049370765686035
training
current loss: 5.993216514587402
training
current loss: 5.941198348999023
training
current loss: 8.575883865356445
training
current loss: 5.1056718826293945
training
current loss: 6.9967122077941895
training
current loss: 7.727817535400391
training
current loss: 6.344146251678467
training
current loss: 7.9867634773254395
training
current loss: 6.991996765136719
training
current loss: 6.850757598876953
training
current loss: 6.573969841003418
training
current loss: 7.0721354484558105
training
current loss: 5.026459217071533
training
current loss: 5.954272270202637
training
current loss: 7.41362190246582
training
current loss: 5.054441452026367
training
current loss: 6.092312335968018
training
current loss: 7.9289445877075195
training
current loss: 6.654376983642578
training
current loss: 6.069939613342285
training
current loss: 6.542503356933594
training
current loss: 8.075883865356445
training
cu

training
current loss: 6.062479019165039
training
current loss: 7.85239839553833
training
current loss: 6.1123809814453125
training
current loss: 6.006380558013916
training
current loss: 7.663303375244141
training
current loss: 5.52585506439209
training
current loss: 6.859793663024902
training
current loss: 6.787890911102295
training
current loss: 5.651032447814941
training
current loss: 5.389668941497803
training
current loss: 7.83926248550415
training
current loss: 6.127013683319092
training
current loss: 5.399109363555908
training
current loss: 5.906515121459961
training
current loss: 6.757706642150879
training
current loss: 8.05049991607666
training
current loss: 7.874599456787109
training
current loss: 7.755462646484375
training
current loss: 8.980405807495117
training
current loss: 6.0217084884643555
training
current loss: 8.376322746276855
training
current loss: 8.128643035888672
training
current loss: 7.026290416717529
training
current loss: 6.992076873779297
training
current l

...

training
current loss: 7.945786476135254
training
current loss: 8.398125648498535
training
current loss: 6.828274726867676
training
current loss: 6.920556545257568
training
current loss: 7.030367374420166
training
current loss: 7.514008522033691
training
current loss: 6.042201042175293
training
current loss: 7.33882474899292
training
current loss: 5.115379810333252
training
current loss: 9.752010345458984
training
current loss: 9.097580909729004
training
current loss: 9.009285926818848
training
current loss: 7.596100807189941
training
current loss: 6.020967960357666
training
current loss: 6.391189098358154
training
current loss: 7.814033508300781
training
current loss: 6.135636329650879
training
current loss: 8.184247970581055
training
current loss: 7.913206100463867
training
current loss: 5.988706111907959
training
current loss: 5.905585289001465
training
current loss: 6.698419570922852
training
current loss: 8.282532691955566
training
current loss: 6.94346284866333
training
current l

training
current loss: 6.304295063018799
training
current loss: 6.350862503051758
training
current loss: 6.230442047119141
training
current loss: 5.299745559692383
training
current loss: 5.015066146850586
training
current loss: 6.0950927734375
training
current loss: 5.049982070922852
training
current loss: 6.256412506103516
training
current loss: 6.020949840545654
training
current loss: 6.05374813079834
training
current loss: 5.654176712036133
training
current loss: 8.06679630279541
training
current loss: 8.366926193237305
training
current loss: 6.502330780029297
training
current loss: 8.024633407592773
training
current loss: 6.255819320678711
training
current loss: 6.838858127593994
training
current loss: 6.6610846519470215
training
current loss: 9.285467147827148
training
current loss: 6.0224504470825195
training
current loss: 10.230299949645996
training
current loss: 9.713037490844727
training
current loss: 7.0232720375061035
training
current loss: 7.402527332305908
training
current

training
current loss: 8.016578674316406
training
current loss: 9.493766784667969
training
current loss: 9.773865699768066
training
current loss: 6.02055549621582
training
current loss: 7.102031707763672
training
current loss: 6.067966938018799
training
current loss: 5.825465202331543
training
current loss: 9.135316848754883
training
current loss: 7.54616641998291
training
current loss: 6.826194763183594
training
current loss: 6.463395118713379
training
current loss: 7.576942443847656
training
current loss: 6.655559539794922
training
current loss: 6.1859893798828125
training
current loss: 6.04295539855957
training
current loss: 5.886505603790283
training
current loss: 6.235899448394775
training
current loss: 6.850043296813965
training
current loss: 5.33441686630249
training
current loss: 6.3492913246154785
training
current loss: 6.2727837562561035
training
current loss: 7.675508975982666
training
current loss: 5.065889835357666
training
current loss: 7.931989669799805
training
current 

training
current loss: 5.198891639709473
training
current loss: 5.100594997406006
training
current loss: 7.293290138244629
training
current loss: 6.222903728485107
training
current loss: 5.0182271003723145
training
current loss: 7.191897869110107
training
current loss: 7.027085304260254
training
current loss: 8.06906509399414
training
current loss: 5.02786922454834
training
current loss: 7.058807373046875
training
current loss: 8.010089874267578
training
current loss: 5.328155040740967
training
current loss: 5.384399890899658
training
current loss: 6.117415428161621
training
current loss: 8.058947563171387
training
current loss: 7.745092391967773
training
current loss: 6.248088836669922
training
current loss: 6.020420551300049
training
current loss: 5.0232930183410645
training
current loss: 7.01708459854126
training
current loss: 6.014987945556641
training
current loss: 6.3214335441589355
training
current loss: 8.405779838562012
training
current loss: 5.014734745025635
training
current

training
current loss: 5.550053119659424
training
current loss: 7.9049530029296875
training
current loss: 7.193953037261963
training
current loss: 7.6959967613220215
training
current loss: 5.0179595947265625
training
current loss: 6.656864643096924
training
current loss: 5.018057823181152
training
current loss: 6.011343955993652
training
current loss: 7.009825706481934
training
current loss: 6.125548839569092
training
current loss: 6.584684371948242
training
current loss: 5.115257740020752
training
current loss: 8.788599014282227
training
current loss: 6.005918979644775
training
current loss: 6.214993953704834
training
current loss: 5.087972640991211
training
current loss: 6.673803329467773
training
current loss: 5.03446626663208
training
current loss: 7.40028715133667
training
current loss: 6.305365562438965
training
current loss: 7.5226545333862305
training
current loss: 5.241548538208008
training
current loss: 5.087096691131592
training
current loss: 7.261316299438477
training
curre

In [25]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1)
            )
          )
          (intermediat